In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from pymongo import MongoClient

In [2]:
df = pd.read_excel("/home/waris/Github/tupras-analysis/alarm.xlsx")
client = MongoClient()

In [3]:
db = client["Tupras-DB"]
collectionT = db["Alarm"]

In [4]:
sources = df["SourceName"].values
start_time = df["ActiveTime"].values
end_time = df["EventTime"].values
message = df["Message"].values
print(sources, start_time, end_time, message)
print(type(end_time[0]), type(message[0]))
print(end_time[0]-start_time[0])


['5FIC366' '18FIC356A' '5FIC316' ... '5PIC373' '5LIC509' '5LIC509'] ['2016-12-20T19:59:28.000000000' '2016-12-30T18:33:54.000000000'
 '2017-01-02T08:24:42.000000000' ... '2017-12-31T23:22:10.000000000'
 '2017-12-31T23:25:30.000000000' '2017-12-31T23:25:30.000000000'] ['2017-01-04T16:45:50.000000000' '2017-01-02T14:18:44.000000000'
 '2017-01-02T08:31:24.000000000' ... '2017-12-31T23:56:50.000000000'
 '2017-12-31T23:25:30.000000000' '2017-12-31T23:25:31.000000000'] ['5FIC366      5E-12 DEN   AGIR DIZEL R ACK'
 '18FIC356A    18G-356A DENPLT5 E H.PET PV  = 219.93 M3/H HI   Recover'
 '5FIC316      5F-3 3.KOL  AKIMI        PV  =  529.7 M3/D LO   Recover'
 ...
 '5PIC373      5C-2 TEPE   BASINCI      MV  =   73.0 %    MHI  Recover'
 '5LIC509      5D-101 MAYI SEVIYESI     PV  =   20.0 %    LO'
 '5LIC509      5D-101 MAYI SEVIYESI     PV  =   37.2 %    LO   Recover']
<class 'numpy.datetime64'> <class 'str'>
1284382000000000 nanoseconds


Preprocessing Step 1

In [16]:
# def getTargetEventList(start,stime,etime):
#     tList = []

#     for i in range(start,len(start_time)):
#         if (stime < start_time[i]) and (start_time[i] < etime) and (end_time[i] < etime):
#             tList.append({"target": sources[i], "target-start-time": np.datetime_as_string(start_time[i]), "target-end-time": np.datetime_as_string(end_time[i])})

#     return tList

In [17]:
# for i, s in enumerate(sources):
#     tList = getTargetEventList(i+1,start_time[i], end_time[i])
#     if len(tList) > 0:
#         print("Iteration", i, "Number of alarms", len(tList))
#         collectionT.insert_one({"source":s, "source-start-time": np.datetime_as_string(start_time[i]), "source-end-time":np.datetime_as_string(end_time[i]),"tList": tList})


ion 61722 Number of alarms 2
Iteration 61723 Number of alarms 7
Iteration 61826 Number of alarms 2
Iteration 61873 Number of alarms 2
Iteration 61874 Number of alarms 2
Iteration 61875 Number of alarms 3
Iteration 61877 Number of alarms 3
Iteration 61878 Number of alarms 1
Iteration 61879 Number of alarms 8
Iteration 61880 Number of alarms 8
Iteration 61881 Number of alarms 8
Iteration 61902 Number of alarms 2
Iteration 61946 Number of alarms 1
Iteration 62005 Number of alarms 30
Iteration 62021 Number of alarms 4
Iteration 62044 Number of alarms 1
Iteration 62050 Number of alarms 91
Iteration 62064 Number of alarms 32
Iteration 62109 Number of alarms 1
Iteration 62125 Number of alarms 34
Iteration 62126 Number of alarms 36
Iteration 62141 Number of alarms 3547
Iteration 62142 Number of alarms 6
Iteration 62189 Number of alarms 104
Iteration 62190 Number of alarms 35
Iteration 62192 Number of alarms 4178
Iteration 62199 Number of alarms 4
Iteration 62207 Number of alarms 67
Iteration 6

In [6]:
from multiprocessing import Pool

def getAndStoreTargetEventList(index):

    tList = []
    s = sources[index]
    start, stime, etime = index+1, start_time[index], end_time[index]

    for i in range(start,len(start_time)):
        # if source[i] == s:
        #     continue
        
        if (stime < start_time[i]):
            if (start_time[i] < etime):
                if (end_time[i] < etime):
                    tList.append({"target": sources[i], "target-start-time": np.datetime_as_string(start_time[i]), "target-end-time": np.datetime_as_string(end_time[i])}) 
            else:
                break
        else:
            break
    
    if len(tList)> 0:
        print("Iteration", index, "Number of alarms", len(tList))
        collectionT.insert_one({"source":s, "source-start-time": np.datetime_as_string(stime), "source-end-time":np.datetime_as_string(etime),"tList": tList})


with Pool(3) as p:
    p.map(getAndStoreTargetEventList, range(len(sources)) )


NameError: name 'source' is not defined